Думаю, вы уже познакомились со стандартными функциями в numpy для вычисления станадртных оценок (среднего, медианы и проч), однако в анализе реальных данных вы, как правило, будете работать с целым датасетом.

В этом разделе мы познакомимся с вычислением описательных статистик для целого датасета.

Большинство из них вычиляются одной командой (методом) describe

С вычислением корреляцонной матрицы мы уже сталкивались во 2 модуле, но освежим и ее

И отдельное внимание уделим вычислению условных и безусловных пропорций.


В датасете framingham.csv представлены данные, которые группа ученых из Фрамингема (США) использовала для выявления риска заболевания ишемической болезнью сердца в течение 10 лет.

Демографические данные: 

    sex (male): пол, мужчина (1) или женщина (0) 

    age: возраст 
    
    education: уровень образования (0-4: школа-колледж)
    
Поведенческие данные: 

    currentSmoker: курильщик (1) или нет (0)

    cigsPerDay: количество выкуриваемых сигарет в день (шт.)

Медицинская история:

    BPMeds: принимает ли пациент препараты для регулировки артериального давления (0 - нет, 1 - да)

    prevalentStroke: случался ли у пациента сердечный приступ (0 - нет, 1 - да)

    prevalentHyp: страдает ли пациент гипертонией (0 - нет, 1 - да)

    diabetes: страдает ли пациент диабетом (0 - нет, 1 - да)

Физическое состояние:

    totChol: уровень холестерина

    sysBP: систолическое (верхнее) артериальное давление 

    diaBP: диастолическое (нижнее) артериальное давление 

    BMI: индекс массы тела - масса (кг) / рост^2 (в метрах)

    heartRate: пульс

    glucose: уровень глюкозы

Целевая переменная (на которую авторы строили регрессию):

    TenYearCHD: риск заболевания ишемической болезнью сердца в течение 10 лет

Импорт библиотек:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import matplotlib.mlab as mlab
%matplotlib inline

Импорт датасета:

In [2]:
df = pd.read_csv('../data/framingham.csv')

Шапка датасета:

In [3]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


Размер датасета:

In [0]:
df.shape

Названия столбцов датасета понадобятся нам, чтобы обращаться к отдельным признакам пациентов:

In [0]:
df.columns

Поиск missing values:

In [0]:
df.isnull().sum()

Видим, что у 105 пациентов нет данных об образовании, у 388 - об уровне глюкозы. Недостающие данные могут стать причиной некорректных значений оценок, и от них лучше избавиться.

Количество строк, в которых есть missing values

In [0]:
count=0
for i in df.isnull().sum(axis=1):
    if i>0:
        count=count+1
print('Общее количество строк с пропущенными значениями: ', count)

Итого у нас 582 строки, в которых не хватает каких-нибудь данных

Найдем соотношение строк с недостающими данными и общего кол-ва строк

Если их будет относительно немного, то выгоднее избавиться от неполных строк, а если много, то надо посмотреть, в каких столбцах больше всего не хватает данных - возможно, выгоднее будет избавляться от них.

In [0]:
count/df.shape[0]

Т.к. соотношение мало (13%), можем отбросить строки с отсутствующими данными):

In [0]:
df.dropna(axis=0,inplace=True)

Размеры датасета после удаления missing values:

In [0]:
df.shape

Получение описательных статистик при помощи фунции describe: 

In [0]:
df.describe()

Данные полученные из describe:

    mean - среднее значение
    std - стандартное (среднеквадратичное) отклонение. 
    min - минимальное значение 
    max - максимальное значение
    25% - нижняя квартиль (медиана меньшей/левой половины выборки)
    50% - медиана
    75% - верхняя квартиль (медиана большей/правой половины выборки)

Далее строим тепловую карту корреляционной матрицы при помощи функции heatmap и саму корреляционную матрицу. Чем насыщеннее цвет, тем сильнее корреляция.

In [0]:
sn.heatmap(df.corr())

Можем увидеть сильную корреляцию между диастолическим и систолическим давлением. Корреляции целевой переменной со всеми признаками невелики. Это значит, что линейная связь между ними очень слабая

In [0]:
df.corr()

Со средним, стандартным отклонением, медианой и корреляцией все ясно.

Давайте выясним, как вычислять выборочные пропорции в датасете

как вычислить долю мужчин в выборке?

Длинный способ: посчитаем количество всех мужчин в выборке при помощи метода value_counts() и поделим его на общее количество пациентов

In [0]:
m=df['male'].value_counts() # счетчик разных значений в dataframe
print("Общее количество мужчин и женщин\n", m)
print("Общее количество мужчин:", m[1])

In [0]:
p_male=m[1]/df.shape[0] # считаем пропорцию мужчин среди всех пациентов
print("Доля мужчин среди всех пациентов:", p_male)

Короткий способ: задать в методе value_counts() специальный параметр, который будет вычислиять не абсолютные частоты (количества), а относительные (пропорции)

In [0]:
df['male'].value_counts(normalize = True ) # параметр normalize = True позволяет считать сразу пропорцию вместо количества

С абсолютными пропорциями тоже ясно. Как насчет условных?

Как вычислить долю курильщиков среди мужчин и среди женщин:

In [0]:
male_groups=df.groupby('male') # groupgy разбивает датасет на группы по признаку пола

Внутри каждой группы можем взять счетчик value_counts() для признака currentSmoker

пол 0 - женщина, пол 1 - мужчина.

In [0]:
male_groups['currentSmoker'].value_counts() # можем отдельно вычислить количество корильщиков среди мужчин и среди женщин

Итак: курит 808 женщин и 981 мужчин

Теперь вычислим пропорции курильщиков внутри каждого пола.

Вы можете убедиться, что это именно условные пропорции, поделив количество курящих мужчин на общее количество мужчин и сравнив результаты, или если заметите, что вероятности внутри каждой группы пола дают в сумме 1

In [0]:
ms=male_groups['currentSmoker'].value_counts(normalize = True) 
print('Доли корильщиков среди мужчин и среди женщин\n',ms)
print('Доля корильщиков среди мужчин:',ms[1,1])

Как вычислить среднее значение пульса у курящих и не курящих:

In [0]:
smok_groups=df.groupby('currentSmoker')

In [0]:
smok_groups['heartRate'].mean()

Как вычислить долю пациентов группы риска среди курящих и не курящих:

In [0]:
srisk=smok_groups['TenYearCHD'].value_counts(normalize = True)
print('Доли группы риска среди курильщиков и не курильщиков\n',srisk)
print('Доля группы риска среди курильщиков:',srisk[1,1])

Трюк по вычислению частот для переменных-индикаторов (значения 1 и 0): сумма значений равна количеству единиц в выборке, а значит, среднее равно доле единиц, то есть частоте:

In [0]:
smok_groups['TenYearCHD'].mean()